In [1]:
# Cài đặt trước: pip install selenium webdriver-manager pandas beautifulsoup4 lxml
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def scrape_itviec_visible():
    # Chrome options cho non-headless (hiện browser)
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Mở full màn hình
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
    
    # Tự động download ChromeDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Ẩn webdriver property
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    jobs_data = []
    
    try:
        print("🔄 Đang mở https://itviec.com/it-jobs...")
        driver.get("https://itviec.com/it-jobs")
        
        # Đợi Cloudflare check (5-15s)
        print("⏳ Đợi Cloudflare bypass (5-15s)...")
        time.sleep(10)
        
        # Đợi jobs load
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-job-id], .job_item, [data-qa='job-listing']")))
        
        print("✅ Page loaded, đang lấy jobs...")
        
        # Scroll để load thêm jobs
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        
        # Tìm job elements (nhiều selector để tương thích)
        job_selectors = [
            "[data-job-id]", 
            ".job_item", 
            "[data-qa='job-listing']",
            "article.job",
            ".jobs-listing__item"
        ]
        
        jobs = []
        for selector in job_selectors:
            jobs = driver.find_elements(By.CSS_SELECTOR, selector)
            if jobs:
                print(f"✅ Tìm thấy {len(jobs)} jobs với selector: {selector}")
                break
        
        if not jobs:
            print("❌ Không tìm thấy jobs, inspect element để check selector mới")
            print("HTML sample:", driver.page_source[:1000])
            return pd.DataFrame()
        
        # Lấy info từng job
        for i, job in enumerate(jobs[:20]):  # Top 20 jobs
            try:
                # Title
                title_elem = job.find_element(By.CSS_SELECTOR, "h3 a, .job_title a, a[data-search--job-selection-job-url], .job__title a")
                title = title_elem.text.strip()
                detail_url = title_elem.get_attribute("href")
                
                # Company
                company_elem = job.find_element(By.CSS_SELECTOR, ".job_company_name a, .company-name, .job__company a")
                company = company_elem.text.strip()
                
                # Location & Salary
                location_elem = job.find_element(By.CSS_SELECTOR, ".job_location, .job__location")
                location = location_elem.text.strip()
                
                # Salary
                salary_elem = job.find_elements(By.CSS_SELECTOR, ".job-salary, .salary")
                salary = salary_elem[0].text.strip() if salary_elem else ""
                
                jobs_data.append({
                    'title': title,
                    'company': company,
                    'location': location,
                    'salary': salary,
                    'url': detail_url
                })
                
                print(f"✅ Job {i+1}: {title[:50]}...")
                
            except Exception as e:
                print(f"⚠️ Skip job {i+1}: {str(e)[:50]}")
                continue
        
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        input("⏸️ Nhấn Enter để đóng browser...")
        driver.quit()
    
    df = pd.DataFrame(jobs_data)
    if not df.empty:
        df.to_csv('itviec_jobs_visible.csv', index=False, encoding='utf-8-sig')
        print(f"💾 Đã lưu {len(df)} jobs vào itviec_jobs_visible.csv")
    
    return df

# Chạy
df = scrape_itviec_visible()
print(df.head())
print(f"Tổng: {len(df)} jobs")


🔄 Đang mở https://itviec.com/it-jobs...
⏳ Đợi Cloudflare bypass (5-15s)...
❌ Error: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x12412d3
	0x1241314
	0x102e6dd
	0x10793a5
	0x107977b
	0x10c0382
	0x109b534
	0x10bdb13
	0x109b2e6
	0x106d321
	0x106e1d4
	0x1495314
	0x14908cb
	0x14ad1aa
	0x125b1d8
	0x12631dd
	0x12495d8
	0x1249799
	0x1233b28
	0x76695d49
	0x773ed5db
	0x773ed561

Empty DataFrame
Columns: []
Index: []
Tổng: 0 jobs
